In [ ]:
# import faiss
import numpy as np
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

# Load CLIP model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Image dataset
image_paths = ["B00O5YHPFU.jpg"]
image_embeddings = []

# Process each image
for img_path in image_paths:
    image = Image.open(img_path)
    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        image_embed = model.get_image_features(**inputs).cpu().numpy()

    image_embeddings.append(image_embed)

# Convert to numpy array
image_embeddings = np.vstack(image_embeddings)

print(image_embeddings)

# # Create FAISS index (L2 distance for similarity)
# index = faiss.IndexFlatL2(image_embeddings.shape[1])
# index.add(image_embeddings)

# # Save index to disk
# faiss.write_index(index, "image_index.faiss")
# np.save("image_paths.npy", np.array(image_paths))  # Save image paths for retrieval


In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import numpy as np
import faiss  # For fast similarity search

# Load CLIP model
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Example dataset of 500 images + texts
images = ["B00O5YHPFU.jpg"]
texts = ["A blue tape wrapped in duct tape"]

# Generate embeddings
image_embeddings = []
text_embeddings = []

for img_path, text in zip(images, texts):
    image = Image.open(img_path)
    inputs = processor(text=[text], images=image, return_tensors="pt", padding=True)

    with torch.no_grad():
        outputs = model(**inputs)
        image_embed = outputs.image_embeds.cpu().numpy()
        text_embed = outputs.text_embeds.cpu().numpy()

    image_embeddings.append(image_embed)
    text_embeddings.append(text_embed)

# Convert to numpy arrays
image_embeddings = np.vstack(image_embeddings)
text_embeddings = np.vstack(text_embeddings)

print(image_embeddings)
print(text_embeddings)

# Store embeddings in FAISS for fast search
index = faiss.IndexFlatL2(image_embeddings.shape[1])  # L2 (Euclidean distance) index
index.add(image_embeddings)  # Store images
faiss.write_index(index, "image_index.faiss")  # Save index

np.save("text_embeddings.npy", text_embeddings)  # Save text embeddings separately


In [ ]:
def search_image(image_path, top_k=5):
    """Search using an image query"""
    image = Image.open(image_path)
    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        query_embed = model.get_image_features(**inputs).cpu().numpy()

    index = faiss.read_index("image_index.faiss")
    distances, indices = index.search(query_embed, top_k)  # Get top K matches
    return indices

def search_text(text_query, top_k=5):
    """Search using a text query"""
    inputs = processor(text=[text_query], return_tensors="pt")

    with torch.no_grad():
        query_embed = model.get_text_features(**inputs).cpu().numpy()

    index = faiss.read_index("image_index.faiss")
    distances, indices = index.search(query_embed, top_k)
    return indices

# Example usage
image_results = search_image("B00O5YHPFU.jpg")
text_results = search_text("dcut tape")
print("Image search results:", image_results)
print("Text search results:", text_results)


In [ ]:
!pip install transformers
!pip install accelerate>=0.26.0
!pip install --upgrade ipywidgets
!pip install --upgrade jupyter
!pip install bitsandbytes
!pip install --upgrade typing_extensions

In [2]:
!pip install faiss

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
pip install faiss-gpu
